In [1]:
# Importing Libraries
import selenium
import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
import warnings
warnings.filterwarnings('ignore')

# SCRAPING DATA OF REVIEWS FROM AMAZON

In [7]:
# Creating python program to search desired keywords one by one and saving links in open list prod_url

# creating open list to save href links of all products 
prod_url = []

# Keywords for search to which we need to scrape the data.
keywords = ['Laptops', 'Phones', 'Headphones', 'Smart Watches', 'Cameras', 'Printers', 'Monitors','Home theater', 'Router']

# Location of chrome web driver and launching it
driver = webdriver.Chrome("C:/Users/User/Downloads/chromedriver_win32 (1)/chromedriver")

# website from which we will scrape data
driver.get("https://www.amazon.in/") 

initial_url = driver.current_url

# loop for sending keywords to website and scraping href
for i in keywords:
    # Finding search input and giving input
    search = driver.find_element_by_id("twotabsearchtextbox").send_keys(i)
        
    # clicking on search button
    search_btn = driver.find_element_by_xpath("//div[@class='nav-search-submit nav-sprite']")
    search_btn.click()
    # giving time to load the webpage completely to avoid errors
    time.sleep(5)
    # finding href of each products mentioned in keywords
    for j in driver.find_elements_by_xpath("//h2[@class = 'a-size-mini a-spacing-none a-color-base s-line-clamp-2']/a"):
        prod_url.append(j.get_attribute('href'))
    # returning to initial url    
    driver.get(initial_url)

In [8]:
# checking length of prod_url
len(prod_url)

176

In [9]:
# opening each 226 prod_url and scraping links of all reviews href 
# open list to save all reviews href
rev_cli = []

# loop for opening all links stored in prod_url
for j in prod_url:
    # opening each links of prod_url one by one
    driver.get(j)
    
    try:
        # finding links from all webpage by xpath
        for k in driver.find_elements_by_xpath("//a[@id = 'acrCustomerReviewLink']"):
            rev_cli.append(k.get_attribute('href'))
    # if all reviews link is not available, it will continue to next page,so avoiding NoSuchElementException 
    except NoSuchElementException as e:
        continue

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=97.0.4692.71)


In [ ]:
# Checking length of open list
len(rev_cli)

In [ ]:
# Now opening each links as stored in list above
# creating open list for storing all reviews href
all_reviews = []

for l in rev_cli:
    # opening each links as stored in cli_rev open list
    driver.get(l)
    time.sleep(3)
    
    try:
        # finding links from all webpage by xpath
        all_rev = driver.find_element_by_xpath("//div[@id = 'reviews-medley-footer']/div[2]/a")
        all_reviews.append(all_rev.get_attribute('href'))
        # if all reviews link is not available, it will continue to next page,so avoiding NoSuchElementException 
    except NoSuchElementException as e:
        continue

In [ ]:
# Checking length of all reviews link
len(all_reviews)

In [ ]:
# finally opening all_reviews link and scraping ratings and reviews data from all webpages.
# creating open list to save scraped data
ratings = []
reviews = []

# opening each link to begin the scaping
for m in all_reviews:
    driver.get(m)
    time.sleep(15)
    
    # loop to open link till page 5 and scraping data of reviews and ratings of products
    for n in range(0,5):
        for o in driver.find_elements_by_xpath("//div[@class = 'a-section review aok-relative']/div/div/div[2]/a[1]"):
            ratings.append(o.get_attribute('title').replace(' out of 5 stars',''))
            
        for p in driver.find_elements_by_xpath("//div[@class = 'a-section review aok-relative']/div/div/div[4]/span/span"):
            reviews.append(p.text)
                        
        try:
            next_page = driver.find_element_by_xpath("//ul[@class = 'a-pagination']/li[2]/a")
            driver.get(next_page.get_attribute('href'))
        except NoSuchElementException as e:
            continue

In [ ]:
# saving scraped data into dataframe.
search = pd.DataFrame({}) 
search['Ratings'] = ratings[:20442]
search['Full_review'] = reviews[:20442]
search

In [ ]:
# Saving scraped data in csv format
search.to_csv('Review_rating.csv')

# Rating Prediction

Importing Libraries

In [ ]:
#importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#importing nltk libraries
import nltk
#nltk.download('all')
from nltk.corpus import stopwords
import re
import string
from nltk import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import FreqDist

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score,precision_score, confusion_matrix, accuracy_score,classification_report

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()

# Data Loading

In [ ]:
#loading the data into a dataframe
df = pd.read_csv("Review_rating.csv")
df

Feature Information:
    
Review_title : title of the review
    
Review_text : content of the review text
    
Ratings : ratings out of 5 stars
    
At first I will drop the unwanted column Unnamed: 0

In [ ]:
df.drop(columns = 'Unnamed: 0',inplace = True)

In [ ]:
#checking the shape
print("Shape :", df.shape)

Great our data set is having 77550 rows and three columns now

In [ ]:
#checking column names
print("Columns :", df.columns)

In [ ]:
#data types
df.dtypes

In [ ]:
#info
df.info()

By checking all above observations we can say the data contains three columns Review_title , Reiew_text and Ratings. All columns are object type. Here Ratings column is our target column.

# Check for missing values

In [ ]:
#missing values
df.isnull().sum()

The data set is having lot of missing values and we will drop these values.

In [ ]:
#Visualization of missing values
sns.set(rc={'figure.figsize':(11.8,8.27)})
sns.heatmap(data=df.isnull())
plt.xticks(rotation=90, fontsize=10)
plt.yticks(rotation=0, fontsize=10)
plt.show()

In [ ]:
#Drop the rows where review_title and Reiew_text has null values
df.dropna(subset=['Review_title','Reiew_text'], inplace=True)

In [ ]:
#checking for null values again
df.isnull().sum()

Great we have successfully removed all the missing data from our data set.

In [ ]:
#lets check the shape of our data set now
df.shape

Lets have a look at our target column

In [ ]:
df['Ratings'].unique()

Looking at these entries in target column we came to know that we need to replace the string entries to respective values(stars)

In [ ]:
df['Ratings'].unique()

# Combine Review_title and Reiew_text into one

In [ ]:
#joining Review text and title
df['Review'] = df['Review_title'].map(str)+' '+df['Reiew_text']

In [ ]:
#lets have a look at our data now
df

Visualizing text in first three rows from Review column

In [ ]:
df['Review'][0]

In [ ]:
df['Review'][1]

In [ ]:
df['Review'][2]

Text Processing

In [ ]:
#Here I am defining a function to replace some of the contracted words to their full form and removing urls and some unwanted text 
def decontracted(text):
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"don’t", "do not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"im ", "i am", text)
    text = re.sub(r"yo ", "you ",text)
    text = re.sub(r"doesn’t", "does not",text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"<br>", " ", text)
    text = re.sub(r'http\S+', '', text) #removing urls
    return text

In [ ]:
#Lowercasing
df['Review'] = df['Review'].apply(lambda x : x.lower())

df['Review'] = df['Review'].apply(lambda x : decontracted(x))

#removing punctuations
df['Review'] = df['Review'].str.replace('[^\w\s]','')
df['Review'] = df['Review'].str.replace('\n',' ')

lets have a look at our text again

In [ ]:
df['Review'][0]

In [ ]:
df['Review'][1]

In [ ]:
#Remove stopwords
stop = stopwords.words('english')
df['Review'] = df['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [ ]:
df['Review'][0]

In [ ]:
df['Review'][1]

We can see that now we are having text without any stop words in it.

# Lemmatization

In [ ]:
#Defining functiom to convert nltk tag to wordnet tags
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [ ]:
#defining function to lemmatize our text
def lemmatize_sentence(sentence):
    #tokenize the sentence & find the pos tag
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    #tuple of (token, wordnet_tag)
    wordnet_tagged = map(lambda x : (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatize_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatize_sentence.append(word)
        else:
            lemmatize_sentence.append(lemmatizer.lemmatize(word,tag))
    return " ".join(lemmatize_sentence)

In [ ]:
df['Review'] = df['Review'].apply(lambda x : lemmatize_sentence(x))

In [ ]:
df['Review'][0]

In [ ]:
df['Review'][1]

Text Normalization - Standardization

In [ ]:
#Noice removal
def scrub_words(text):
    #remove html markup
    text = re.sub("(<.*?>)", "", text)
    #remove non-ascii and digits
    text = re.sub("(\\W)", " ", text)
    text = re.sub("(\\d)", "", text)
    #remove white space
    text = text.strip()
    return text

In [ ]:
df['Review'] = df['Review'].apply(lambda x : scrub_words(x))

In [ ]:
df['Review'][0]

In [ ]:
df['Review'][1]

Finally I have defined a function scrub_words for removing the noise from the text. It will remove any html markups, digits and white spaces from the text. We can understand it by looking at first two row's text from review column

Now We have did all the text-processing steps and got required input for our model. We will go for EDA now

# Exploratory Data Analysis

Word Counts

In [ ]:
#creating column for word counts in the text
df['Review_WC'] = df['Review'].apply(lambda x: len(str(x).split(' ')))
df[['Review_WC','Review']].head()

In [ ]:
#density plot and histogram of all word count
sns.distplot(df['Review_WC'], hist = True, kde = True,
            bins = int(180/5), color = 'darkblue',
            hist_kws = {'edgecolor':'black'},
            kde_kws = {'linewidth':4})
plt.show()

Looking at the above histogram we can clearly observe that most of our text is having the number of words in the range of 0 to 200, But some of the reviews are too lengthy which may act like outliers in our data.

# Character count

In [ ]:
#creating column for character counts in the text
df['Review_CC'] = df['Review'].str.len()
df[['Review_CC','Review']].head()

In [ ]:
#density plot and histogram of all character count
sns.distplot(df['Review_CC'], hist = True, kde = True,
            bins = int(180/5), color = 'darkblue',
            hist_kws = {'edgecolor':'black'},
            kde_kws = {'linewidth':4})
plt.show()

Above plot represents histogram for character count of review text, which is quite similar to the histogram of word count.

Removing Outliers

As we know that some of the review are too lengthy I am removing those reviews from the dats as outliers using z_score method.

Great, by removing outliers we are loosing only arround 1000 entries which is acceptable here for getting beter results for our model.

# Plotting histograms for word count and character counts again after removing outliers

In [ ]:
#density plot and histogram of all word count
sns.distplot(df['Review_WC'], hist = True, kde = True,
            bins = int(180/5), color = 'darkblue',
            hist_kws = {'edgecolor':'black'},
            kde_kws = {'linewidth':4})
plt.show()

In [ ]:
#density plot and histogram of all word count
sns.distplot(df['Review_CC'], hist = True, kde = True,
            bins = int(180/5), color = 'darkblue',
            hist_kws = {'edgecolor':'black'},
            kde_kws = {'linewidth':4})
plt.show()

After plotting histograms for word counts and character counts after removing outliers we can see now we are with good range of number of words and characters

In [ ]:
#lets have a look at our data set
df

In [ ]:
#Check the count of target column
sns.countplot(df['Ratings'])
print(df.Ratings.value_counts())
plt.show()

Looking at the above count plot for our target varible(Ratings) we can say that the data set is having most number of reviews rated as 5 star. and very less number of reviews rated as 2.

Which will cause the Imbalance problem for our model.

So I am selecting equal number of reviews of each rating as a input for our model

For that first I will shuffle the dataset so that we can select data from both web-sites

Then I will select equal number of data of every category

In [ ]:
#shuffling the data set
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
df

In [ ]:
#selct data from every category
df1 = df[df['Ratings']==1][0:7356]
df2 = df[df['Ratings']==2][0:7356]
df3 = df[df['Ratings']==3][0:7356]
df4 = df[df['Ratings']==4][0:7356]
df5 = df[df['Ratings']==5][0:7356]

In [ ]:
#Combining all the dataframes into one and shuffling them 
df = pd.concat([df1,df2,df3,df4,df5],ignore_index=True)
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
#lets Check the count of target column again
sns.countplot(df['Ratings'])
print(df.Ratings.value_counts())
plt.show()

Great, we have balanced our data.

Top 30 most frequently occuring words

In [ ]:
#function to plot most frequent terms
def freq_words(x, terms = 30):
    all_words = ' '.join([text for text in x])
    all_words = all_words.split()
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()),
                             'count':list(fdist.values())})
    #selecting top 30 most freq words
    d = words_df.nlargest(columns = 'count', n = terms)
    plt.figure(figsize = (20,10))
    ax = sns.barplot(data = d, x='count', y='word')
    ax.set(ylabel = 'Word')
    plt.show()

In [ ]:
freq_words(df['Review'])

# Top 30 Rare words

In [ ]:
#function to plot least frequent terms
def rare_words(x, terms = 30):
    all_words = ' '.join([text for text in x])
    all_words = all_words.split()
    fdist = FreqDist(all_words)
    words_df = pd.DataFrame({'word':list(fdist.keys()),
                             'count':list(fdist.values())})
    #selecting top 30 most freq words
    d = words_df.nsmallest(columns = 'count', n = terms)
    plt.figure(figsize = (20,10))
    ax = sns.barplot(data = d, x='count', y='word')
    ax.set(ylabel = 'Word')
    plt.show()

In [ ]:
rare_words(df['Review'])

In [ ]:
!pip install wordcloud

Word cloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)
def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
                    background_color='white',
                    stopwords = stopwords,
                    max_words = 500,
                    max_font_size = 40,
                    scale = 3,
                    random_state = 1).generate(str(data))
    fig = plt.figure(1, figsize=(15,15))
    plt.axis('off')
    if title:
        fig.suptitle(title, fontsize=20)
        fig.subplots_adjust(top=2.3)
    plt.imshow(wordcloud)
    plt.show()

Words for rating

In [ ]:
show_wordcloud(df1['Review'])

In [ ]:
show_wordcloud(df2['Review'])

In [ ]:
show_wordcloud(df3['Review'])

In [ ]:
show_wordcloud(df4['Review'])

In [ ]:
show_wordcloud(df5['Review'])

# Model Building and Evaluation

In [ ]:
#separating features and labels
x = df['Review']
y = df['Ratings']

# Converting text into vectors using TfidfVectorizer

In [ ]:
#using the n_gram tfidf vectorizer(Word vectors)
word_vectorizer = TfidfVectorizer(
                                sublinear_tf = True,
                                strip_accents = 'unicode',
                                analyzer = 'word',
                                token_pattern = r'\w{1,}',
                                stop_words = 'english',
                                ngram_range = (1,3),
                                max_features = 100000)
word_vectorizer.fit(x)
train_word_features = word_vectorizer.transform(x)

In [ ]:
#Character vectors
char_vectorizer = TfidfVectorizer(
                                sublinear_tf = True,
                                strip_accents = 'unicode',
                                analyzer = 'char',
                                stop_words = 'english',
                                ngram_range = (2,6),
                                max_features = 50000)
char_vectorizer.fit(x)
train_char_features = char_vectorizer.transform(x)

In [ ]:
#I will combine both word vectors and character vectors as input for our model
train_features = hstack([train_char_features,train_word_features])

Splitting the data into train and test

In [ ]:
seed = 1
x_train, x_test, y_train, y_test = train_test_split(train_features, y, test_size = 0.25, random_state = seed)

In [ ]:
#lets check the shapes of traning and test data
print("x_train", x_train.shape)
print("x_test", x_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)

In [ ]:
#defining the algorithms
rf = RandomForestClassifier()
svc = LinearSVC()
lr = LogisticRegression(solver='lbfgs')
mnb = MultinomialNB()
xgb = XGBClassifier(verbosity=0)
bnb = BernoulliNB()
lgb = LGBMClassifier()
sgd = SGDClassifier()

In [ ]:
#creating a function to train and test the model with evaluation
def BuiltModel(model):
    print('*'*30+model.__class__.__name__+'*'*30)
    model.fit(x_train,y_train)
    y_pred = model.predict(x_train)
    pred = model.predict(x_test)

    accuracy = accuracy_score(y_test,pred)*100

    print(f"Accuracy Score:", accuracy)
    print("---------------------------------------------------")

    #confusion matrix & classification report
    
    print(f"CLASSIFICATION REPORT : \n {classification_report(y_test,pred)}")
    print(f"Confusion Matrix : \n {confusion_matrix(y_test,pred)}\n")

Training and testing of various algorithms

In [ ]:
for model in [lr,svc,bnb,mnb,sgd,rf,xgb]:
    BuiltModel(model)

Great, among all these algorithms 4 are giving good accuracies: LogisticRegression, LinearSVC, SGDClassifier and RandomForestClassifier

Cross validation score

In [ ]:
def cross_val(model):
    print('*'*30+model.__class__.__name__+'*'*30)
    scores = cross_val_score(model,train_features,y, cv = 3).mean()*100
    print("Cross validation score :", scores)


I am checking cross-validation score only for those algorithms which are giving us better accuracies

In [ ]:
for model in [lr,svc,sgd,rf]:
    cross_val(model)

Great all our algorithms are giving good cv scores with very less difference in accuracy and cv-scores. Among these algorithms I am selecting LinearSVC as best suitable algorithm for our final model as it is giving least difference in accuracy and cv score with higher accuracy.

HyperParameter Tuning

In [ ]:
#lets selects different parameters for tuning
grid_params = {
                'penalty':['l2'],
                'loss':['hinge','squared_hinge'],
                'multi_class': ['ovr'],
                'intercept_scaling':[2,3],
                'dual':[True,False],
                }

In [ ]:
#train the model with given parameters using GridSearchCV
GCV =  GridSearchCV(svc, grid_params, cv = 3, verbose=10)
GCV.fit(x_train,y_train)

In [ ]:
GCV.best_params_       #printing the best parameters found by GridSearchCV

# Final Model

In [ ]:
#training and testing our final model with above parameters
model = LinearSVC(dual = True, intercept_scaling = 2, loss = 'hinge', multi_class = 'ovr', penalty = 'l2')
model.fit(x_train,y_train) #fitting data to model
pred = model.predict(x_test)
accuracy = accuracy_score(y_test,pred)*100

#printing accuracy score
print("Accuracy Score :", accuracy)

#printing Confusion matrix
print(f"\nConfusion Matrix : \n {confusion_matrix(y_test,pred)}\n")

#printing Classification report
print(f"\nCLASSIFICATION REPORT : \n {classification_report(y_test,pred)}")

Great; after doing hyperparameter tuning we have got improved accuracy score for our final model.

# Model Saving

In [ ]:
import joblib
joblib.dump(model,"Rating_Prediction.pkl")

Finally I am saving this model into a .pkl file